In [ ]:
import cv2
import serial
import time

# Hubungkan ke ESP32
try:
    ser = serial.Serial('COM6', 115200, timeout=1)  # Ganti COM sesuai port kamu
    time.sleep(2)
    print("ESP32 connected.")
except Exception as e:
    print("Gagal konek ke ESP32:", e)
    ser = None

# === MODE KAMERA ===
def kamera_mode():
    cap = cv2.VideoCapture(0)
    print("\nMode OTOMATIS (kamera).")
    print("Tekan 'm' → manual | 'q' → keluar")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        # Ambil bagian tengah
        h_center = frame.shape[0] // 2
        w_center = frame.shape[1] // 2
        center = frame[h_center-50:h_center+50, w_center-50:w_center+50]

        # Gambar kotak di frame utama
        cv2.rectangle(frame, (w_center-50, h_center-50), (w_center+50, h_center+50), (255, 255, 255), 2)

        # Konversi ke HSV
        hsv = cv2.cvtColor(center, cv2.COLOR_BGR2HSV)
        avg_color = hsv.mean(axis=0).mean(axis=0)
        h, s, v = avg_color

        command = None
        color = "UNKNOWN"

        # Deteksi warna
        if 35 < h < 85:
            color = "GREEN"
            command = b'1'
        elif 20 < h <= 35:
            color = "YELLOW"
            command = b'3'
        elif 85 < h <= 130:
            color = "BLUE"
            command = b'2'
        elif h <= 10 or h >= 160:
            color = "RED"
            command = b'0'
        elif s < 30 and v > 200:
            color = "WHITE"
            command = b'4'

        if ser and command:
            ser.write(command)

        # Tampilkan deteksi warna
        cv2.putText(frame, f"COLOR: {color}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0) if color == "GREEN" else
                    (0, 255, 255) if color == "YELLOW" else
                    (255, 0, 0) if color == "BLUE" else
                    (255, 255, 255) if color == "WHITE" else
                    (0, 0, 255), 2)

        # Tampilkan frame dan crop tengah
        cv2.imshow("Camera View", frame)
        cv2.imshow("Center View", center)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return 'quit'
        elif key == ord('m'):
            cap.release()
            cv2.destroyAllWindows()
            return 'manual'

# === MODE INPUT ===
def manual_mode():
    print("\nMode MANUAL.")
    print("Ketik:")
    print("1 → MAJU | 2 → KANAN | 3 → KIRI | 4 → MUNDUR | 0 → STOP")
    print("'a' → kamera | 'q' → keluar")

    while True:
        cmd = input("Input: ").strip().lower()

        if cmd == 'q':
            return 'quit'
        elif cmd == 'a':
            return 'kamera'
        elif cmd in ['0', '1', '2', '3', '4']:
            if ser:
                ser.write(cmd.encode())
        else:
            print("Perintah tidak valid!")

# === PILIH MODE AWAL ===
mode = input("Pilih mode awal: [a] Kamera / [m] Manual: ").lower()

# === LOOP UTAMA ===
while True:
    if mode == 'a':
        next_mode = kamera_mode()
    elif mode == 'm':
        next_mode = manual_mode()
    else:
        print("Mode tidak dikenali.")
        break

    # Normalisasi nama mode
    if next_mode == 'kamera':
        mode = 'a'
    elif next_mode == 'manual':
        mode = 'm'
    elif next_mode == 'quit':
        print("Keluar dari program.")
        break
    else:
        print("Mode tidak dikenali.")
        break

# === Tutup koneksi ===
if ser:
    ser.close()


Gagal konek ke ESP32: could not open port 'COM6': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)


Pilih mode awal: [a] Kamera / [m] Manual:  a



Mode OTOMATIS (kamera).
Tekan 'm' → manual | 'q' → keluar

Mode MANUAL.
Ketik:
1 → MAJU | 2 → KANAN | 3 → KIRI | 4 → MUNDUR | 0 → STOP
'a' → kamera | 'q' → keluar


Input:  1
Input:  3
Input:  4
Input:  0


Input:  1
Input:  2
Input:  3
Input:  4
Input:  5


Perintah tidak valid!


Input:  0
Input:  a



Mode OTOMATIS (kamera).
Tekan 'm' → manual | 'q' → keluar
Keluar dari program.


Pilih mode awal: [a] Kamera / [m] Manual:  a


Mode OTOMATIS aktif (kamera). Tekan 'm' untuk mode manual, 'q' untuk keluar.
Mode tidak dikenali.
